<a href="https://colab.research.google.com/github/amazingsmash/FOM_Facial_Animation/blob/master/Facial_Animations_with_%22First_Order_Motion_Model%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo for paper "First Order Motion Model for Image Animation"

Cloning repository with data

In [1]:
!git clone https://github.com/amazingsmash/FOM_Facial_Animation.git

Cloning into 'FOM_Facial_Animation'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


**Clone repository and install all the requirements**


Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing to your google drive.



In [2]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model
!cd first-order-model

from google.colab import drive
drive.mount('/content/gdrive')

Cloning into 'first-order-model'...
remote: Enumerating objects: 193, done.
remote: Total 193 (delta 0), reused 0 (delta 0), pack-reused 193
Receiving objects: 100% (193/193), 71.45 MiB | 10.05 MiB/s, done.
Resolving deltas: 100% (95/95), done.


**Create a model and load checkpoints**

In [0]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

**Load driving video and source image**

In [0]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

**Perform image animation**

In [0]:
from demo import make_animation
from skimage import img_as_ubyte

source_image = imageio.imread('/content/FOM_Facial_Animation/target_face_2.png')
driving_video = imageio.mimread('/content/FOM_Facial_Animation/driving_video_trump.mp4', memtest=False)

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('/generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

anim = display(source_image, driving_video, predictions)

HTML(anim.to_html5_video())

anim.save("/content/FOM_Facial_Animation/trump_to_face1.mp4")

100%|██████████| 211/211 [00:09<00:00, 22.23it/s]


In [0]:
source_image = imageio.imread('/content/FOM_Facial_Animation/target_face_2.png')
driving_video = imageio.mimread('/content/FOM_Facial_Animation/driving_video_trump.mp4', memtest=False)

#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 211/211 [00:09<00:00, 22.05it/s]


In [0]:
source_image = imageio.imread('/content/FOM_Facial_Animation/target_face_sanchez.png')
driving_video = imageio.mimread('/content/FOM_Facial_Animation/driving_video_1.mp4', memtest=False)


#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

anim = display(source_image, driving_video, predictions)

anim.save("driving_face1_to_sanchez.mp4")

HTML(anim.to_html5_video())

100%|██████████| 120/120 [00:05<00:00, 22.12it/s]


In [0]:
source_image = imageio.imread('/content/FOM_Facial_Animation/target_face_sanchez.png')
driving_video = imageio.mimread('/content/FOM_Facial_Animation/driving_video_2.mp4', memtest=False)


#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

anim = display(source_image, driving_video, predictions)

anim.save("driving_face2_to_sanchez.mp4")

HTML(anim.to_html5_video())

100%|██████████| 154/154 [00:06<00:00, 22.03it/s]


Commands of interest for video and audio 

In [0]:
#Transcode and crop center of video to 256x256
!ffmpeg -i in.mov -vf "crop=in_h*1/1:in_h,scale=-2:256" out.mp4 

#Extract audio 
!ffmpeg -i in.mp4 -f mp3 -ab 192000 -vn out.mp3   

#Change audio speed to match lenghts with video
ffmpeg -i in.mp3 -filter:a "atempo=1.25" out.mp3

#Join audio and video
ffmpeg -i video.mp4 -i audio.mp3 -shortest -c:v copy -c:a aac -b:a 256k output.mp4
